# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [5]:
import os
import openai
# import getpass

# Dotenv library to load configuration from .env file
import dotenv
dotenvfile = dotenv.find_dotenv()
PROJECT_ROOT = os.path.dirname(dotenvfile)  # Use dotenv to find root of the project

# Load configuration from .env file
dotenv.load_dotenv(dotenvfile)

openai.api_key = os.environ["OPENAI_API_KEY"]
#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")
#print(os.environ["OPENAI_API_KEY"])

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

---
**ANSWER:**

Qdrant, along with other vector databases, are made for storing and querying high-dimensional vectors efficiently, using specialized data structures and indexes. These databases enable fast similarity and semantic search while allowing users to find vectors that are the closest to a given query vector based on some distance metric. 

Qdrant, specifically, uses:

- Hierarchical Navigable Small World (HNSW), which is used to implement Approximate Nearest Neighbors. HNSW is a graph-based indexing algorithm, which builds a multi-layer navigation structure according to certain rules. The upper layers are more sparse and the distances between nodes are farther. The lower layers are denser and the distances between nodes are closer. The search starts from the uppermost layer, finds the node closest to the target in this layer, and then enters the next layer to begin another search. This is repeated until the target is approached.
- Compression techniques such as Product Quantization, Scalar Quantization and Binary Quantization, which enable efficient storage and search of high-dimensional vectors by transforming the original vectors into new representations with less precision with minimal loss in precision.

In addition, Qdrant has a few other features that make it a good choice for a Vector database, such as:
- Multi-Vector Support: Qdrant allows the integration of multiple vectors per document. This enables more nuanced and relevant search results by combining different aspects of the data, such as the title and body of a document.
- Option to store vectors in memory or on-disk, allowin gthe user to decide on the trade-off of speed vs storage capacity
- Distributed deployment, to distribute the data across the peers to extend the storage capabilities and increase stability.

---

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [12]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [13]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [14]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '28cf674d6adc4fbebebb238d471b649d', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [16]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. 
If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [18]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

---
**ANSWER:**

In short, the pipeline takes the question asked by the user and runs it through the vector store retrieval process to get the top-n most similar documents to the question as the context. Both the question and the context (as a dictionary) are passed forward to the prompt template, which formats the prompt, and then forwarded to the primary QA LLM (in this case gpt-3.5-turbo), so that it returns a response. The context is also provided in the final dictionary, unchanged.

A step-by-step description of the pipeline is this:

The pipeline starts with creating a dictionary with two keys, "context" and "question". The question is simply the same string informed by the user, which is forwarded to the chain via the `invoke` method. The "context" key, on the other hand, is created by passing the original question through the "retriever" function, which has been previously defined as a Qdrand vector store retriever. This will search in the vector store for the most similar documents to the question and assign it to the "context" key. These steps happen in parallel. 

Next, the RunnablePassthrough step allows the question to be passed forward unchanged, and the context gets assigned to the dictionary as well.

Finally, both keys (context and question) are passed to the promt template for proper formatting and then to the LLM, which will generate the response. The result will be a dictionary with the response from the LLM and the retrieved context.
  
  
---

Let's test it out!

In [19]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [20]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '6e6f1b1ac1894673bf8ec36952c6d058', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [21]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

---
**ANSWER:**

This is important because if the document chunks are exactly the same as the original chunks used for setting up the RAG pipeline, we risk having synthetic questions that will end up not evaluating our RAG properly. This is because, as mentioned in the Ragas documentation, "LLMs by default are not good at creating diverse samples as it tends to follow common paths". If our evaluation questions are exactly the same as the documents in the RAG pipeline, we will end up not being able to create diverse and hard enough questions that will test the RAG pipeline in conditions that would be most similar to a production environment. It will be something akin to overfitting, as our evaluation will only show us how well the RAG pipeline works with questions extracted from the exact same environment in which the RAG was trained.

(review)

---

In [22]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [26]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo") #<--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
#critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the four major forms of chance and th...,[that chance is immune from human intervention...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does the concept of commitment relate to o...,[you can reply to a lot of messages with “I’m ...,The concept of commitment relates to organizin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are some criteria to consider when evalua...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,Why does the author believe that intelligence ...,[Criteria 7rst\nLots of people will tell you t...,The author believes that intelligence is overr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What aspects of the startup culture in the Val...,[would have spent 10 or 20 or 30 years buildin...,The culture of startups in the Valley contribu...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What are the potential consequences of not bei...,[Here’s why you shouldn’t do that:\nWhat are t...,The potential consequences of not being able t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How did cheap bandwidth contribute to the grow...,"[late 90’s — due to commodity hardware, open s...",Cheap bandwidth in the late 90's contributed t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the significance of a large market in ...,[billion people online now. That is a very lar...,A large market in the context of the internet ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How can a transformative deal inject energy ba...,"[cut some big deals, do some spinoWs, whatever...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does living a structured existence affect ...,[Arnold Schwarzenegger’s success as a movie st...,Living a structured existence can limit one's ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

---
**ANSWER:**

This mapping shows our desired proportion of the different types of questions to be generated by the algorithm. Currently, Ragas has the following types of "evolutions" that generate these different types of questions: "Simple", "Reasoning", "Conditioning" and "Multi-Context".

- Simple: Simple questions.
- Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.
- Conditioning: Modify the question to introduce a conditional element, which adds complexity to the question.
- Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.

---

Let's look at the output and see what we can learn about it!

In [27]:
testset.test_data[0]

DataRow(question='What are the four major forms of chance and their reasons in human interactions and creative reactions?', contexts=['that chance is immune from human interventions. However, one\nmust be careful not to read any unconsciously purposeful intent\ninto “interventions”… [which] are to be viewed as accidental,\nunwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…\nWe can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:\nThe four kinds of chance each have a diWerent kind of motor'], ground_truth='nan', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page':

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [28]:
test_df = testset.to_pandas()

In [29]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the four major forms of chance and th...,[that chance is immune from human intervention...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does the concept of commitment relate to o...,[you can reply to a lot of messages with “I’m ...,The concept of commitment relates to organizin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are some criteria to consider when evalua...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,Why does the author believe that intelligence ...,[Criteria 7rst\nLots of people will tell you t...,The author believes that intelligence is overr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What aspects of the startup culture in the Val...,[would have spent 10 or 20 or 30 years buildin...,The culture of startups in the Valley contribu...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What are the potential consequences of not bei...,[Here’s why you shouldn’t do that:\nWhat are t...,The potential consequences of not being able t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How did cheap bandwidth contribute to the grow...,"[late 90’s — due to commodity hardware, open s...",Cheap bandwidth in the late 90's contributed t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the significance of a large market in ...,[billion people online now. That is a very lar...,A large market in the context of the internet ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How can a transformative deal inject energy ba...,"[cut some big deals, do some spinoWs, whatever...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How does living a structured existence affect ...,[Arnold Schwarzenegger’s success as a movie st...,Living a structured existence can limit one's ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [30]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [31]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [32]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [33]:
response_dataset[0]

{'question': 'What are the four major forms of chance and their reasons in human interactions and creative reactions?',
 'answer': "I don't know",
 'contexts': ['We can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:',
  'unwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…',
  'The four kinds of chance each have a diWerent kind of motor\nexploratory activity and a diWerent kind of sensory receptivity.\nThe [four] varieties of chance also involve distinctive personality',
  'Luck and the entrepreneur: The four kinds of luck\n163'],
 'ground_truth': 'nan'}

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [34]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [36]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

No statements were generated from the answer.
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/python3.11/site-packages/ragas/executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/python3.11/site-packages/ragas/executor.py", line 83, in _aresults
    raise e
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/mbrandao/miniconda3/envs/aie3-w4d2/lib/pytho

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
results

In [ ]:
results_df = results.to_pandas()
results_df

## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [ ]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [ ]:
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [ ]:
print(response["answer"])

Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [ ]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [ ]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

In [ ]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [ ]:
results

And see how our advanced retrieval modified our chain!

In [ ]:
advanced_retrieval_results

In [ ]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [ ]:
# Creates the object that access the OpenAI Embeddings API, selecting which embedding model to use
# Here we're selecting a different embedding model to test how much better is it
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# Initiates the Qdrant vector store and loads the documents, in memory
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [ ]:
# Creates a LangChain retriever from the vector store
new_retriever = vector_store.as_retriever()

In [ ]:
# Creates a MultiQueryRetriever that is able to fetch a more diverse set of documents from the vector store
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [ ]:
# Run the retrieval chain that retrieves documents and then passes them on. (function from LangChain)
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [ ]:
# For each of the test question in our test set, invoke the retrieval chain and get the response from the llm.
# Append the answers and the contexts to a list

answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [ ]:
# Convert the new questions/answers/contexts/ground truth into a dataset
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
# Run the function to evaluate the new results
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

In [ ]:
# Print the results
new_advanced_retrieval_results

In [25]:
# Format the results for visualization
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

NameError: name 'pd' is not defined

####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE

primary_qa_llm_gpt4o = ChatOpenAI(model_name="gpt-4o", temperature=0)
gpt4o_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm_gpt4o)
gpt4o_retrieval_chain = create_retrieval_chain(gpt4o_advanced_retriever, document_chain)

answers_gpt4o = []
contexts_gpt4o = []

for question in test_questions:
  response = gpt4o_retrieval_chain.invoke({"input" : question})
  answers_gpt4o.append(response["answer"])
  contexts_gpt4o.append([context.page_content for context in response["context"]])

gpt4o_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers_gpt4o,
    "contexts" : contexts_gpt4o,
    "ground_truth" : test_groundtruths
})    

gpt4o_advanced_retrieval_results = evaluate(gpt4o_response_dataset_advanced_retrieval, metrics)

print(new_advanced_retrieval_results)

In [ ]:
# Format the results for visualization
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])
df_comparison2 = pd.DataFrame(list(gpt4o_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR (GPT-4o)'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")
df_merged = pd.merge(df_merged, df_comparison2, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']
df_merged['Baseline -> TE3 + MQR (GPT-4o)'] = df_merged['TE3 + MQR (GPT-4o)'] - df_merged['ADA + Baseline']

df_merged